<a href="https://colab.research.google.com/github/muslikhannur/P_SalesData_WranglingTest/blob/master/WranglingTest_SalesData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Mount Drive***

In [1]:
# Mount!
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# ***Load Libraries***

In [2]:
# Import some mainstream Library 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time 

from numpy import hstack 
from functools import reduce
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# ***Determine Open/Save Directory***

In [3]:
# Determine the Path and Retrieve the Data
pathDatasets = '/content/drive/My Drive/Colab/Projects - Niagahoster/Datasets/' #Getting Datasets
pathResults = '/content/drive/My Drive/Colab/Projects - Niagahoster/Results/' #Saving Excel, Image, etc

SalesData = pd.read_csv(pathDatasets + 'NH_Technical_Test.csv')
SalesData.tail()

,client_id,invoice_id,invoice_item_id,invoice_status,service_status,product_id,monthly_price_usd,discount,service_period,client_task,invoice_created_at,invoice_paid_at,service_cancelled_at,monthly_period,total_price,discounted_price
89540,259942,881975.0,884574,Paid,Active,3,13,0.50,1Y,Activate,2019-12-16T06:14:36Z,2019-12-18T06:21:09Z,NaN,12,156,78.00
89541,84615,882013.0,1070069,Paid,Active,3,13,0.50,1Y,Renew,2019-12-16T07:00:45Z,2019-12-20T03:20:26Z,NaN,12,156,78.00
89542,129895,882042.0,1155193,Paid,Active,4,18,0.37,1Y,Renew,2019-12-16T07:24:39Z,2019-12-16T10:04:59Z,NaN,12,216,136.08
89543,259997,882147.0,952162,Paid,Active,3,13,0.75,3Y,Activate,2019-12-16T09:17:21Z,2019-12-16T09:20:06Z,NaN,36,468,117.00
89544,251782,882187.0,920682,Paid,Active,3,13,0.78,3Y,Activate,2019-12-16T10:23:09Z,2019-12-16T10:26:42Z,NaN,36,468,102.96


# ***Count Distint/Count Value per Column***

In [4]:
# Count Distinct
cntDistinct = SalesData.nunique() #Count distict values
cntCount = SalesData.count() #Count only non-null values
cntDistinct = pd.DataFrame(data=cntDistinct)
cntCount = pd.DataFrame(data=cntCount)
cntDistinct['col'], cntCount['col']= cntDistinct.index, cntCount.index

cntMerge = [cntDistinct, cntCount]
cntMerge = reduce(lambda left,right: pd.merge(left,right,on='col',how='left'), cntMerge)
cntMerge = cntMerge[['col', '0_x', '0_y']]
cntMerge.columns = ['col', 'distinct', 'count']
cntMerge

,col,distinct,count
0,client_id,56211,89545
1,invoice_id,85681,89545
2,invoice_item_id,88890,89545
3,invoice_status,4,89545
4,service_status,5,89545
5,product_id,14,89545
6,monthly_price_usd,14,89545
7,discount,224,89545
8,service_period,6,89545
9,client_task,2,89545


# ***Question A***
Imagine you are working as a data analyst working at company "X". You were asked to make an annual report by the product manager. In order to do that, you need data which you might already have right now. However, you were informed by the data engineer that there was a problem recently in the company's database. As a result, some records are believed to be duplicated and this would create a huge amount of error in your report.  As a data analyst, it is also part of your job to do some data cleaning. Therefore, separate all the duplicated records, count how many of them, and remove them completely from your data so that you can feel confident about the validity of your analysis.

In [5]:
# Get to know the Duplicate: Based on invoice_id & product_id that have same value
SalesDataDuplicate = SalesData[SalesData.duplicated(['invoice_id', 'product_id'])]
SalesDataDuplicate.tail()

,client_id,invoice_id,invoice_item_id,invoice_status,service_status,product_id,monthly_price_usd,discount,service_period,client_task,invoice_created_at,invoice_paid_at,service_cancelled_at,monthly_period,total_price,discounted_price
89540,259942,881975.0,884574,Paid,Active,3,13,0.50,1Y,Activate,2019-12-16T06:14:36Z,2019-12-18T06:21:09Z,NaN,12,156,78.00
89541,84615,882013.0,1070069,Paid,Active,3,13,0.50,1Y,Renew,2019-12-16T07:00:45Z,2019-12-20T03:20:26Z,NaN,12,156,78.00
89542,129895,882042.0,1155193,Paid,Active,4,18,0.37,1Y,Renew,2019-12-16T07:24:39Z,2019-12-16T10:04:59Z,NaN,12,216,136.08
89543,259997,882147.0,952162,Paid,Active,3,13,0.75,3Y,Activate,2019-12-16T09:17:21Z,2019-12-16T09:20:06Z,NaN,36,468,117.00
89544,251782,882187.0,920682,Paid,Active,3,13,0.78,3Y,Activate,2019-12-16T10:23:09Z,2019-12-16T10:26:42Z,NaN,36,468,102.96


In [6]:
# Sampling to getting to know the Duplicate, Except First Row
SalesDataDuplicate.loc[SalesDataDuplicate['invoice_id'] == 879438.0] #3
SalesData.loc[SalesData['invoice_id'] == 879438.0] #4

,client_id,invoice_id,invoice_item_id,invoice_status,service_status,product_id,monthly_price_usd,discount,service_period,client_task,invoice_created_at,invoice_paid_at,service_cancelled_at,monthly_period,total_price,discounted_price
128,247837,879438.0,960520,Paid,Active,3,13,0.78,3Y,Activate,2019-12-13T11:55:39Z,2019-12-13T11:58:23Z,NaN,36,468,102.96
515,247837,879438.0,988495,Paid,Cancelled,3,13,0.75,3Y,Activate,2019-12-13T11:55:39Z,2019-12-13T11:58:23Z,NaN,36,468,117.00
51552,247837,879438.0,1229821,Paid,Active,3,13,0.78,3Y,Activate,2019-12-13T11:55:39Z,2019-12-13T11:58:23Z,NaN,36,468,102.96
51553,247837,879438.0,1229822,Paid,Cancelled,3,13,0.75,3Y,Activate,2019-12-13T11:55:39Z,2019-12-13T11:58:23Z,NaN,36,468,117.00


In [7]:
# Start Cleaning
SalesDataClean = SalesData.drop(list(list(SalesDataDuplicate.index)))
SalesDataClean.tail()

,client_id,invoice_id,invoice_item_id,invoice_status,service_status,product_id,monthly_price_usd,discount,service_period,client_task,invoice_created_at,invoice_paid_at,service_cancelled_at,monthly_period,total_price,discounted_price
88217,167962,882294.0,1234042,Unpaid,Pending Setup,1,6,0.29,1Y,Activate,2019-12-16T15:50:32Z,NaN,NaN,12,72,51.12
88218,154829,882303.0,1234057,Unpaid,Pending Setup,4,18,0.42,1Y,Activate,2019-12-16T16:17:52Z,NaN,NaN,12,216,125.28
88219,260103,882308.0,1234065,Unpaid,Pending Setup,1,6,0.32,1Y,Activate,2019-12-16T16:30:42Z,NaN,NaN,12,72,48.96
88220,123628,882310.0,1234067,Unpaid,Active,2,9,0.31,1Y,Activate,2019-12-16T16:36:42Z,NaN,NaN,12,108,74.52
88221,167962,882312.0,1234070,Unpaid,Pending Setup,1,6,0.29,1Y,Activate,2019-12-16T16:42:36Z,NaN,NaN,12,72,51.12


In [8]:
# Conclusion in Number
print("So, \nThe number of Records in Raw Sales Data: " + str(len(SalesData))
      + "\nThe number of Records in Duplicated Sales Data: " + str(len(SalesDataDuplicate))
      + "\nThe number of Records in Cleaned Sales Data: " + str(len(SalesDataClean)))

So, 
The number of Records in Raw Sales Data: 89545
The number of Records in Duplicated Sales Data: 3718
The number of Records in Cleaned Sales Data: 85827


# ***Question B***
What's the total number of sales along with the total revenue shared from each product? <br> Arrange the result by the total revenue in descending order.

In [9]:
# Status Exploration
print(SalesData.invoice_status.unique())
print(SalesData.service_status.unique())

['Paid' 'Refunded' 'Cancelled' 'Unpaid']
['Active' 'Suspended' 'Cancelled' 'Failed Setup' 'Pending Setup']


For Question B up to Question F we need to eliminate All the Records with invoice_status Refunded, Cancelled, Unpaid since we can't do sales analysis if the customer itself has not bought a Product

In [10]:
# Start eliminating those who hass not Bought or not in PAID invoice_status
SalesDataCleanPaid = SalesDataClean.loc[SalesDataClean['invoice_status'] == 'Paid']
SalesDataCleanPaid.tail()

,client_id,invoice_id,invoice_item_id,invoice_status,service_status,product_id,monthly_price_usd,discount,service_period,client_task,invoice_created_at,invoice_paid_at,service_cancelled_at,monthly_period,total_price,discounted_price
52423,74461,876940.0,1225832,Paid,Active,3,13,0.50,3Y,Renew,2019-12-11T00:00:41Z,2019-12-31T02:57:17Z,NaN,36,468,234.00
52426,15425,880099.0,1230653,Paid,Active,3,13,0.50,1Y,Renew,2019-12-13T22:12:57Z,2019-12-31T09:03:16Z,NaN,12,156,78.00
52427,50642,877841.0,1227281,Paid,Active,3,13,0.50,1Y,Renew,2019-12-11T20:24:53Z,2019-12-31T09:37:55Z,NaN,12,156,78.00
52429,69925,881656.0,1232953,Paid,Active,12,49,0.11,1M,Renew,2019-12-15T20:12:53Z,2019-12-31T10:45:22Z,NaN,1,49,43.61
78426,235710,798135.0,1104274,Paid,Pending Setup,3,13,0.50,1Y,Activate,2019-09-17T04:11:42Z,NaN,NaN,12,156,78.00


# ***Question C***
The company is planning to give free vouchers to its loyal customers at the end of the year. <br> How do you define "loyal" in this case and explain why? <br> List 100 customers who are eligible to get free vouchers based on your definition of "loyal".

# ***Question D***
What's the average time duration for clients to complete their payments? <br> Hint: you may want to check the distribution of all those time durations to choose a correct central-tendency measure

# ***Question E***
How would you segment the company's customers based on their purchase history? <br> While you are allowed to use your own intuition, you should always support your argument using the data available.

# ***Question F***
Explore the data. State your interesting findings as well as recommendation (if any) to the product team.